In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [2]:
%cd
from DeepLearning.Project2.data_loading_preparation import *

/home/kacper


In [3]:
def get_generic_classifier(input_size, hidden_size, output_size):
    classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size),
    )
    return classifier
    
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cuda'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs)
            pred = torch.argmax(yhat, dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [5]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [6]:
def run_experiment(experiment_name, train_func, run, train_params=None):
    path = f"experiments_rnn/{experiment_name}_run_{run}_"
    print("Running experiment for ", path[:-1])
    
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(train_params)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": experiment_name,
                "train_params": train_params,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [7]:
training_device = "cuda"
device = "cuda"
max_epochs = 250

def noaugLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_validation(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


def timeshiftLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_timeshift(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


def pitchshiftLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_pitchshift(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


def speedshiftLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_speedshift(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


def timemaskLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_timemask(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

def freqmaskLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_freqmask(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


def timeshiftfreqmaskLSTMdifferentiator(params):  
    train, test, val = load_audio_dataloaders_timeshift_freqmask(bs=128, limit_11=0.0)
    criterion_weights = torch.tensor([1.] * 11 + [0.0]).to(device)
    model = LSTMSimple(**params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy

In [8]:
param_grid = {
    "smallsingle": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "mediumsingle": {
        "hidden_size": 128,
        "input_size": 20,
        "num_layers": 1,
    },
    "smallsinglefatclassifier": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
        "classifier_size": 1024
    },
    "smalldoublefatclassifier": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 2,
        "classifier_size": 1024
    },
    "mediumsinglefatclassifier": {
        "hidden_size": 128,
        "input_size": 20,
        "num_layers": 1,
        "classifier_size": 1024
    },
}


experiment_list = [
    (
        f"trainer_{trainer.__name__}_params_{param_name}", 
        trainer,
        str(run),
        param
    )
    for run in range(1, 11) 
    for trainer in [noaugLSTMdifferentiator, pitchshiftLSTMdifferentiator, speedshiftLSTMdifferentiator, timeshiftLSTMdifferentiator, timemaskLSTMdifferentiator, freqmaskLSTMdifferentiator, timeshiftfreqmaskLSTMdifferentiator]
    for param_name, param in param_grid.items()
]

In [9]:
len(experiment_list)

350

In [10]:
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        "Experiment:", experiment[0]
    )
    try:
        run_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-04-21T18:00:08.333797 Experiment: trainer_noaugLSTMdifferentiator_params_smallsingle
Running experiment for  experiments_rnn/trainer_noaugLSTMdifferentiator_params_smallsingle_run_1
Report exists already for experiments_rnn/trainer_noaugLSTMdifferentiator_params_smallsingle_run_1. Skipping...
Time: 2023-04-21T18:00:08.334142 Experiment: trainer_noaugLSTMdifferentiator_params_mediumsingle
Running experiment for  experiments_rnn/trainer_noaugLSTMdifferentiator_params_mediumsingle_run_1
Report exists already for experiments_rnn/trainer_noaugLSTMdifferentiator_params_mediumsingle_run_1. Skipping...
Time: 2023-04-21T18:00:08.334166 Experiment: trainer_noaugLSTMdifferentiator_params_smallsinglefatclassifier
Running experiment for  experiments_rnn/trainer_noaugLSTMdifferentiator_params_smallsinglefatclassifier_run_1
Report exists already for experiments_rnn/trainer_noaugLSTMdifferentiator_params_smallsinglefatclassifier_run_1. Skipping...
Time: 2023-04-21T18:00:08.334186 Experiment

Epoch: 1, Accuracy on validation set: 0.31627906976744186
Epoch: 2, Accuracy on validation set: 0.3926356589147287
Epoch: 3, Accuracy on validation set: 0.437984496124031
Epoch: 4, Accuracy on validation set: 0.4387596899224806
Epoch: 5, Accuracy on validation set: 0.5031007751937985
Epoch: 6, Accuracy on validation set: 0.5189922480620155
Epoch: 7, Accuracy on validation set: 0.5251937984496124
Epoch: 8, Accuracy on validation set: 0.5662790697674419
Epoch: 9, Accuracy on validation set: 0.5833333333333334
Epoch: 10, Accuracy on validation set: 0.5678294573643411
Epoch: 11, Accuracy on validation set: 0.5581395348837209
Epoch: 12, Accuracy on validation set: 0.6174418604651163
Epoch: 13, Accuracy on validation set: 0.5895348837209302
Epoch: 14, Accuracy on validation set: 0.612015503875969
Epoch: 15, Accuracy on validation set: 0.6344961240310077
Epoch: 16, Accuracy on validation set: 0.6348837209302326
Epoch: 17, Accuracy on validation set: 0.6058139534883721
Epoch: 18, Accuracy on v

Epoch: 46, Accuracy on validation set: 0.6282945736434109
Epoch: 47, Accuracy on validation set: 0.6686046511627907
Epoch: 48, Accuracy on validation set: 0.6496124031007752
Epoch: 49, Accuracy on validation set: 0.6581395348837209
Epoch: 50, Accuracy on validation set: 0.6546511627906977
Epoch: 51, Accuracy on validation set: 0.663953488372093
Epoch: 52, Accuracy on validation set: 0.6689922480620155
Epoch: 53, Accuracy on validation set: 0.6573643410852713
Epoch: 54, Accuracy on validation set: 0.622093023255814
Epoch: 55, Accuracy on validation set: 0.6387596899224807
Epoch: 56, Accuracy on validation set: 0.5988372093023255
Epoch: 57, Accuracy on validation set: 0.637984496124031
Epoch: 58, Accuracy on validation set: 0.6875968992248062
Early stop
Time: 2023-04-21T18:14:45.414868 Experiment: trainer_freqmaskLSTMdifferentiator_params_smallsinglefatclassifier
Running experiment for  experiments_rnn/trainer_freqmaskLSTMdifferentiator_params_smallsinglefatclassifier_run_4
Epoch: 1, Acc

Epoch: 26, Accuracy on validation set: 0.5790697674418605
Epoch: 27, Accuracy on validation set: 0.560077519379845
Epoch: 28, Accuracy on validation set: 0.5585271317829458
Epoch: 29, Accuracy on validation set: 0.6170542635658914
Epoch: 30, Accuracy on validation set: 0.625968992248062
Epoch: 31, Accuracy on validation set: 0.6166666666666667
Epoch: 32, Accuracy on validation set: 0.560077519379845
Epoch: 33, Accuracy on validation set: 0.5589147286821705
Epoch: 34, Accuracy on validation set: 0.5135658914728682
Epoch: 35, Accuracy on validation set: 0.5313953488372093
Epoch: 36, Accuracy on validation set: 0.5624031007751938
Epoch: 37, Accuracy on validation set: 0.6131782945736434
Epoch: 38, Accuracy on validation set: 0.5864341085271317
Epoch: 39, Accuracy on validation set: 0.560077519379845
Epoch: 40, Accuracy on validation set: 0.5887596899224806
Epoch: 41, Accuracy on validation set: 0.5717054263565892
Epoch: 42, Accuracy on validation set: 0.6108527131782946
Epoch: 43, Accurac


KeyboardInterrupt

